### Implementing ReAct AgentExecutor

In [7]:
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from langchain.tools.tavily_search import TavilySearchResults
from langchain_community.utilities.tavily_search import TavilySearchAPIWrapper
from langchain_core.prompts import PromptTemplate
from langchain_core.tools import tool
from langchain import hub
from langchain.agents import create_react_agent

In [4]:
load_dotenv()
search = TavilySearchAPIWrapper()
tavily_tool = TavilySearchResults(api_wrapper=search, max_results=3)
llm = ChatGroq(model="llama-3.3-70b-versatile")
react_prompt: PromptTemplate = hub.pull("hwchase17/react")

In [ ]:
# Implementing simple tool
@tool
def do_triple(num: float) -> float:
    """
    :param num: a number to triple
    :return: the number tripled ->  multiplied by 3    
    """    
    return (float(num)*3)

In [ ]:
# Combining multiple tools
tools = [tavily_tool, do_triple]

In [ ]:
# Creating ReAct agent
react_agent = create_react_agent(llm=llm, tools=tools, prompt=react_prompt)